# 17/05 - Pair Programming ETL 1 - Lidia y Tatiana

Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

In [ ]:
url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

En este caso os recomendamos que uséis el producto meteo para obtener la información climática.
 
Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el dataset.

Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. 

Pero antes de poneros manos a la obra, tenemos muchísimos paises y esto puede hacerse eterno. Sacad la información climática solo para los siguientes paises, con las siguientes coordenadas:

👉🏻 Requisitos de este ejercicio de pair programming:

- Deberéis meter toda la información climática en un único dataframe.

- Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que interar la lista de paises y sacar la información del clima para cada uno de ellos.

- Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del pais.

- Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡

Crearos un diccionario donde:

Las keys sean los nombres de los paises

Los values sean las coordenadas de los paises en formato lista.

Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura:

In [11]:
pais= "USA","Australia","South Africa","New Zealand","Papua New Guinea"
latitud_longitud= [39.7837304,-100.445882], [-24.7761086,134.755],[-28.8166236,24.991639], [-41.5000831,172.8344077], [-5.6816069,144.2489081]

diccionario = dict(zip(pais, latitud_longitud))
diccionario

{'USA': [39.7837304, -100.445882],
 'Australia': [-24.7761086, 134.755],
 'South Africa': [-28.8166236, 24.991639],
 'New Zealand': [-41.5000831, 172.8344077],
 'Papua New Guinea': [-5.6816069, 144.2489081]}

In [133]:
#Creamos un diccionario vacio
df_completo= pd.DataFrame()

for key, value in diccionario.items():

    lon= value[1]
    lat= value[0]
    producto= "meteo"
    url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json' 

    response = requests.get(url=url)
    response.status_code
    response.reason

    #Creamos el dataFrame que viene de la API
    df_API = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
    
    #Le decimos que añada las columas según nuestro diccionario
    df_API["pais"]=key
    df_API["latitud"]=value[0]
    df_API["longitud"]=value[1]
    
    #Unimos los data set
    df_completo = pd.concat([df_completo,df_API], axis = 0)


In [134]:
df_completo.sample(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
37,114,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,-1,11,1014,none,2,0,95,3,Papua New Guinea,-5.681607,144.248908
40,123,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 255, 'speed':...",18,2,10,1020,none,8,0,250,3,South Africa,-28.816624,24.991639


In [135]:
df_completo.to_csv("df_completo.csv")